
# Atelier 4 : Classification


# 1.	Objectif : 

L’objectif de cet atelier est de découvrir la classification de documents texte à travers le classificateur SVM que nous allons appliquer sur un dataset contenant les news apparus sur le fil de presse Reuters en 1987.

Le datset peut être téléchargé à partir de ce lien : https://archive.ics.uci.edu/ml/datasets/reuters-21578+text+categorization+collection. 

Le dataset est disponible egalement sur nltk :


In [1]:
import nltk
nltk.download('reuters')

[nltk_data] Downloading package reuters to
[nltk_data]     C:\Users\ismail\AppData\Roaming\nltk_data...


True

Le dataset est disponible également sur scikitlearn 


In [ ]:
from sklearn.datasets import fetch_rcv1

In [11]:
classes_Annotated_coffee=reuters.categories(reuters.fileids("coffee"))


# 2.	Chargement des données 

Nous considerons le dataset reuters à partir de NLTK.

In [13]:
from nltk.corpus import reuters
#recuperation du vocabulaire du corpus
vocabulaire=reuters.words()

#recuperation de toutes les categories
categories=reuters.categories()

#recuperation de tous les id des fichiers appartenant à une categorie bien determinée
ids_coffe=reuters.fileids("coffee")

#recuperation des mots contenus dans les documents d'une categorie bien determinee
coffe_words=reuters.words(reuters.fileids("coffee"))

#recuperation du texte brut des documents d'une categorie bien determinee
cofee_docs=reuters.raw(reuters.fileids("coffee")[0])

#recuperation de toutes les autres classe d'un document annoté avec une classe bien determinee
classes_Annotated_coffee=reuters.categories(reuters.fileids("coffee"))

#recuperer le dataset d'apprentissage
train_categories=[ reuters.categories(i) for i in reuters.fileids() if i.startswith('training/')]
train_documents = [reuters.raw(i) for i in reuters.fileids() if i.startswith('training/')]

#recuperer le dataset de test
test_documents=[reuters.raw(i)  for i in reuters.fileids() if i.startswith('test/')]
test_categories = [reuters.categories(i) for i in reuters.fileids() if i.startswith('test/')]

# recuperer tout le corpus
whole_docs=[reuters.raw(i)for i in reuters.fileids()]
whole_cats = [ reuters.categories(i) for i in reuters.fileids()]


In [15]:
#TF-IDF
#fit(raw_documents[, y]): Learn vocabulary and idf from training set.
#fit_transform(raw_documents[, y]): Learn vocabulary and idf, return document-term matrix.
#transform(raw_documents): Transform documents to document-term matrix.

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(stop_words = 'english')

vect_whole_docs = vectorizer.fit_transform(whole_docs)
vect_train_docs = vectorizer.transform(train_documents)
vect_test_docs = vectorizer.transform(test_documents)

#recuperer des labels uniques pour les categories
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
train_labels = mlb.fit_transform(train_categories)
test_labels = mlb.transform(test_categories)
whole_labels = mlb.fit_transform(whole_cats)

print(whole_labels.shape)

(10788, 90)


# 4.	Classification avec SVM

In [19]:
#SVM
import numpy as np
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.metrics import classification_report

classifier_svm = OneVsRestClassifier(LinearSVC())
classifier_svm.fit(vect_train_docs,train_labels)
test_labels_predict=classifier_svm.predict(vect_test_docs)
print(classification_report(test_labels,test_labels_predict))
scores=classifier_svm.score(vect_test_docs,test_labels)
print(scores)

              precision    recall  f1-score   support

           0       0.99      0.95      0.97       719
           1       1.00      0.43      0.61        23
           2       1.00      0.64      0.78        14
           3       0.95      0.60      0.73        30
           4       0.88      0.39      0.54        18
           5       0.00      0.00      0.00         1
           6       1.00      0.94      0.97        18
           7       1.00      0.50      0.67         2
           8       0.00      0.00      0.00         3
           9       0.96      0.96      0.96        28
          10       1.00      0.78      0.88        18
          11       0.00      0.00      0.00         1
          12       0.95      0.71      0.82        56
          13       1.00      0.50      0.67        20
          14       0.00      0.00      0.00         2
          15       0.92      0.43      0.59        28
          16       0.00      0.00      0.00         1
          17       0.91    

C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 5.	Classification  SVM avec cross validation

In [20]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.metrics import recall_score
scoring = ['precision_samples', 'recall_samples', 'f1_samples']
scores_svm = cross_validate(classifier_svm, vect_whole_docs, whole_labels, cv=3, scoring=scoring)
print(scores_svm['fit_time'])
print(scores_svm['score_time'])
print(scores_svm['test_precision_samples'])
print(scores_svm['test_recall_samples'])
print(scores_svm['test_f1_samples'])




C:\Users\ismail\anaconda3\lib\site-packages\sklearn\multiclass.py:81: UserWarning: Label not 5 is present in all training examples.
  str(classes[c]))
C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


[6.00917673 6.15166998 4.85522914]
[0.43297696 0.4484303  0.32841825]
[0.86598999 0.87977382 0.88399611]
[0.84546102 0.86121606 0.86284829]
[0.8475128  0.86216063 0.86600443]


C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# 6.	Exercice


Q1. Réaliser le même processus avec le KNN et comparer les performances obtenues avec SVM.

Q2. Réaliser le même processus avec une méthode ensembliste et comparer les performances obtenues avec KNN et SVM. 

NB: There is no multi-label version of AdaBoost and the label should be the shape of samples number. 



## Q1

In [45]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=9)
knn.fit(vect_train_docs,train_labels)

KNeighborsClassifier(n_neighbors=9)

In [46]:
test_labels_predict=knn.predict(vect_test_docs)
print(classification_report(test_labels,test_labels_predict))
scores=knn.score(vect_test_docs,test_labels)
print(scores)

C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      0.70      0.81       719
           1       1.00      0.35      0.52        23
           2       1.00      0.36      0.53        14
           3       0.55      0.60      0.57        30
           4       0.69      0.50      0.58        18
           5       0.00      0.00      0.00         1
           6       0.94      0.89      0.91        18
           7       0.00      0.00      0.00         2
           8       0.00      0.00      0.00         3
           9       0.88      0.82      0.85        28
          10       0.82      0.78      0.80        18
          11       0.00      0.00      0.00         1
          12       0.77      0.61      0.68        56
          13       1.00      0.45      0.62        20
          14       0.00      0.00      0.00         2
          15       0.79      0.54      0.64        28
          16       0.00      0.00      0.00         1
          17       0.83    

In [47]:
print(scores)

0.7316992381583306


nous remarquons que le svm est bien meilleur que le  knn ceci revient au fait que des colonnes peuvent etre correler entre eux ce qui va diminuer la performance du knn

## Q2

In [50]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.multiclass import OneVsRestClassifier
bc = OneVsRestClassifier(GradientBoostingClassifier())
bc.fit(vect_train_docs,train_labels)
#boostPreds = boostClassifier.predict(vect_test_docs)
scores=bc.score(vect_test_docs,test_labels)


In [51]:
scores

0.7419675389201722

In [52]:
print(classification_report(test_labels,bc.predict(vect_test_docs)))


              precision    recall  f1-score   support

           0       0.96      0.86      0.91       719
           1       0.92      0.52      0.67        23
           2       0.85      0.79      0.81        14
           3       0.61      0.47      0.53        30
           4       0.68      0.72      0.70        18
           5       0.00      0.00      0.00         1
           6       1.00      1.00      1.00        18
           7       1.00      1.00      1.00         2
           8       0.00      0.00      0.00         3
           9       0.90      0.93      0.91        28
          10       1.00      0.78      0.88        18
          11       0.00      0.00      0.00         1
          12       0.88      0.89      0.88        56
          13       0.88      0.75      0.81        20
          14       0.00      0.00      0.00         2
          15       0.60      0.54      0.57        28
          16       0.00      0.00      0.00         1
          17       0.88    

C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\ismail\anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1248: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


Nous pouvons que pour la macro qui traite le scoring de chaque classe individuellement des autres la méthode de gradientboosting est bien meilleur que le svm mais en macro avg et les autres  SVM depasse cette methode  la SVM reste un des modeles les plus performant du fait qu'elle maximise les marges qui sépare un groupe de la frontiére ceci rend l'apprentissage plus adéquat